## Scope of the Project

PyMC3 runs much of its computation using [Theano](http://deeplearning.net/software/theano/). As Theano is no longer going to be maintained, the developers of PyMC3 have expressed interest in allowing for multiple symbolic backends for pymc3. This venture has been started as PyMC4 (see this [discussion thread](https://discourse.pymc.io/t/tensorflow-backend-for-pymc4/409/15)).

Within this github repo, I altered some of the PyMC3 source code so that it can run on either Theano or [TensorFlow](https://www.tensorflow.org/) backends. [Keras](https://keras.io/) is a package with similar functionality, and I used it as a model (see backends [documenation](https://keras.io/backend/) or the [source code](https://github.com/keras-team/keras/tree/master/keras/backend) for Keras).

Note: The term 'symbolic backends' here differs from 'backends' in pymc3, which are different storage options (hdf5, ndarrays etc.) for MCMC traces. 


In [1]:
from utils import display_code


## Changes to the PyMC3 Code Base

I've only converted enough code, so that I can recreate this basic example ["A Motivating Example: Linear Regression"](http://docs.pymc.io/notebooks/getting_started#A-Motivating-Example:-Linear-Regression). However, the some principles that I used to make pymc3 (almost) agnostic to symbolic backend, can hopefully be extended to the rest of the code base. 

The pymc3 code base is called 'pymc3-master', and consists of these modules: 

In [2]:
!ls ./pymc3-master/pymc3

__init__.py       distributions     memoize.py        theanof.py
__pycache__       examples          model.py          tuning
backends          exceptions.py     plots             util.py
backends_symbolic external          sampling.py       variational
blocking.py       glm               stats.py          vartypes.py
data.py           gp                step_methods
diagnostics.py    math.py           tests


### Overview of code changed

In order to run the basic example, I needed to alter the following modules: 
- `__init__.py`
- `model.py`
- `distributions/distribution.py`
- `distributions/continuous.py`
- `step_methods/arraystep.py`
- `step_methods/metropolis.py`
- `step_methods/compound.py`


However, I also heavily **commented code** that I did not want to change yet. For example, I only kept the `Normal` distribution in `continuous.py`, and commented all the rest out. 



Much of the Theano functionality is either directly imported from Theano at the top of each module, or imported from
pymc3 modules such as `theanof.py` or `math.py`. I commented these imports and instead import all the necessary functionality from my new module called `symbolic_backends`.


In [13]:
display_code('model.py',[8,20])

Within `symbolic_backends`, I have implementations of the same functions in two different modules, `backends_tf.py` and `backends_theano.py`.


In [9]:
!ls ./pymc3-master/pymc3/*backends_symbolic*

__init__.py        backends_tf.py     common.py
__pycache__        backends_theano.py


The default symbolic backend is theano, but TensorFlow can be specified by setting an environment variable `os.environ["PYMC_SYMB_BACKEND"] = "tensorflow" `


In [16]:
file = 'backends_symbolic/__init__.py'
lines=[4,20]
display_code(file,lines)

### Changes needed for model.py

Pymc3 models consist of classes random variables. The main two are ObservedRV and FreeRV, which do and do not have observed data, respectively. The classes were originally TensorVariables in Theano. 


#### TensorVariables
They now differ depending on backend. It's still a `TensorVariable` in Theano, but is a `Variable` in TensorFlow. 

Note that I left a `##CHANGED:` comment everywhere that I adjusted the original pymc3 code. 

In [19]:
file = 'model.py'
lines=[1234,1239]
display_code(file,lines)

In [28]:
file = 'backends_symbolic/backends_tf.py'
lines=[1,17]
display_code(file,lines)

One implication of this change is that using a TensorFlow backend requires an `initial_value` be specified for all variables, even free random variables. 

There are a few other tensor variable related functions that I needed to implement for both backends. I've added doc strings, but these functions are pretty self-explanatory.

In [32]:
file = 'backends_symbolic/backends_tf.py'
lines=[25,50]
display_code(file,lines)

In [45]:
file = 'backends_symbolic/backends_theano.py'
lines=[18,45]
display_code(file,lines)

### Changes needed for distribution.py

Free and Observed random variables in pymc come with `Distributions` classes. I only altered the `Normal` distribution subclass of the `Continuous` distribution class. Distributions require values from variables, for example: 

#### Getting Values from Symbolic Variables

In [42]:
file = 'distributions/distribution.py'
lines=[86,100]
display_code(file,lines)

Getting values differs in Theano and TensorFlow. In Theano, if a variable has a value, it can be obtained directly. The type of variable needs to be checked beforehand, however, because the call is slightly different. 

In [47]:
file = 'backends_symbolic/backends_theano.py'
lines=[45,56]
display_code(file,lines)

In TensorFlow, a variable needs to be fed to a session and the session needs to be run in order to return a value. 


In [52]:
file = 'backends_symbolic/backends_tf.py'
lines=[54,60]
display_code(file,lines)

In TensorFlow, a variable needs to be fed to a session and the session needs to be run in order to return a value. 

#### TensorFlow Sessions

The TensorFlow backend deals with instantiating a session the first time around and retrieving it when needed

In [55]:
file = 'backends_symbolic/backends_tf.py'
lines=[80,130]
display_code(file,lines)

#### Functions

Another major change was to replace the calls to `theano.function`, with a more agnostic implementation. Keras' implementation helped a lot here. 

For example, in a function that draws values from a distribution, I changed `_compile_theano_function` which was basically a memoized and wrapped version of `theano.function`, which defines a function and compiles it. 


In [67]:
file = 'distributions/distribution.py'
lines=[398,405]
display_code(file,lines)

Instead, like keras, I've wrapped `theano.function` into a class called `Function`, which gets instantiated by the a `function` function. The initialization compiles the function, and the call evaluates the function. 

In [70]:
file = 'backends_symbolic/backends_theano.py'
lines=[97,145]
display_code(file,lines)

I did this inorder to use the same syntax for TensorFlow, which doesn't compile a function per se. Here, the intialization relates the inputs, outputs, updates, and session variable. The call runs the session. 


In [77]:
file = 'backends_symbolic/backends_tf.py'
lines=[130,215]
display_code(file,lines)

This `Function` class is used for when pymc's `Model` class compiles the likelihood function using the `logp` output variables and the various input variables. 


In [79]:
file = 'model.py'
lines=[917,943]
display_code(file,lines)

### Changes needed for continuous.py

The `Normal` distribution class contains the parameters of the distribution. 

In [56]:
file = 'distributions/continuous.py'
lines=[296,304]
display_code(file,lines)

And the symbolic definition of the loglikelihood. These are the outputs in the computation graph and with the inputs will be be compiled into a Theano function or fed to a TensorFlow session. 

In [59]:
file = 'distributions/continuous.py'
lines=[322,331]
display_code(file,lines)

#### Basic Math
All of the basic mathematical functions required for specifying the likelihood have been moved to the backend. 

In [64]:
file = 'backends_symbolic/backends_tf.py'
lines=[220,234]
display_code(file,lines)

### Changes needed for arraystep.py

The MCMC sampling in pymc is done by taking what's called a 'step' in parameter space from the current parameter values, a 'point', to a new set of values. The `ArrayStepShared` class takes one step for each parameter at a time. 

Each parameter then has it's own instantiation of a particular step method, for example the Metropolis method. Therefore this `step` function gets called for each variable in the model. 

The simplest solution for now was to write a conditional for the different backends. In theano, we only pass one input variable to the step method. In TensorFlow, I'm passing all the variables. 

In [89]:
file = 'step_methods/arraystep.py'
lines=[162,184]
display_code(file,lines)

### Changes needed for metropolis.py

The metropolis method works as follows: 
- For a particular variable, and random step in parameter step is proposed from some proposal distribution (usually a normal distribution). 
- This proposed step `delta` is added to the original parameter value `q0`, 
- Then the change in the models log likelihood, `delta_logp`, is calculated.
- If the change in likelihood is greater than some threshold, the new parameter value `q` is accepted. 

In [100]:
file = 'step_methods/metropolis.py'
lines=[163,164]
display_code(file,lines)

In [108]:
file = 'step_methods/metropolis.py'
lines=[178,182]
display_code(file,lines)

I've moved the calculation of the change in log likelihood to the symbolic backend. This compiled function gets associated with the metroplis step method upon its instantiation for each variable.  


In [99]:
file = 'backends_symbolic/backends_theano.py'
lines=[312,330]
display_code(file,lines)

For TensorFlow, I've implemented it slightly differently at the moment. Instead of having a `delta_logp` function, I just call the `logp` function twice, once for the old and new values, and calculate the difference. This is slower, but was an easier first pass. 

In [118]:
file = 'step_methods/metropolis.py'
lines=[184,204]
display_code(file,lines)

Again, both of these functions rely on the `Function` class. 

In [125]:
file = 'backends_symbolic/backends_tf.py'
lines=[283,294]
display_code(file,lines)

## Test Case: A Motivating Example: Linear Regression

### Running with original pymc3 code (restart kernel)

Here I import an unaltered PyMC3 package. 

In [8]:
import sys
import numpy as np
sys.path.append('/Users/chris/anaconda/envs/env_pymc3/lib/python3.6/')
sys.path.append('/Users/chris/anaconda/envs/env_pymc3/lib/python3.6/site-packages/')
import pymc3 as pm

In [9]:
# Initialize random number generator
np.random.seed(123)

# True parameter values
alpha, sigma = 1, 1; beta = [1, 2.5]

# Predictor variable
X1 = np.random.randn(100)
X2 = np.random.randn(100) * 0.2

# Simulate outcome variable
Y = alpha + beta[0]*X1 + beta[1]*X2 + np.random.randn(100)*sigma
Y=Y.astype('float32')

In [10]:
basic_model2 = pm.Model()
with basic_model2:
    alpha = pm.Normal(name='alpha',mu=0.0, sd=10.0) 
    beta = pm.Normal(name='beta', mu=0.0, sd=10.0, shape=2)

    mu = alpha + beta[0]*X1 + beta[1]*X2

    Y_obs = pm.Normal(name='Y_obs',mu=mu, sd=1.0, observed=Y)

    step = pm.Metropolis()
    trace = pm.sample(5000,step=step,cores=1,chains=1) # switched off parallel for simplifications. 

print(np.mean(trace['alpha']))
print(np.mean(trace['beta'],axis=0))

Sequential sampling (1 chains in 1 job)
CompoundStep
>Metropolis: [beta]
>Metropolis: [alpha]
100%|██████████| 5500/5500 [00:01<00:00, 3819.80it/s]
Only one chain was sampled, this makes it impossible to run some convergence checks


0.9054529
[0.9458999 2.6006136]


### Running with theano backend (restart kernel for a fresh import)

Here I import the local PyMC3 package that I changed. I set the backend to be theano. 

In [1]:
import sys
import os
import numpy as np
os.environ["PYMC_SYMB_BACKEND"] = "theano" 
sys.path.append('./pymc3-master/')
import pymc3 as pm

Using Theano backend.
/Users/chris/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Initialize random number generator
np.random.seed(123)

# True parameter values
alpha, sigma = 1, 1; beta = [1, 2.5]

# Predictor variable
X1 = np.random.randn(100)
X2 = np.random.randn(100) * 0.2

# Simulate outcome variable
Y = alpha + beta[0]*X1 + beta[1]*X2 + np.random.randn(100)*sigma
Y=Y.astype('float32')

In [3]:
basic_model2 = pm.Model()
with basic_model2:
    alpha = pm.Normal(name='alpha',mu=0.0, sd=10.0) 
    beta = pm.Normal(name='beta', mu=0.0, sd=10.0, shape=2)

    mu = alpha + beta[0]*X1 + beta[1]*X2

    Y_obs = pm.Normal(name='Y_obs',mu=mu, sd=1.0, observed=Y)

    step = pm.Metropolis()
    trace = pm.sample(5000,step=step,cores=1,chains=1) # switched off parallel for simplifications. 

print(np.mean(trace['alpha']))
print(np.mean(trace['beta'],axis=0))

Sequential sampling (1 chains in 1 job)
CompoundStep
>Metropolis: [beta]
>Metropolis: [alpha]
100%|██████████| 5500/5500 [00:02<00:00, 2738.67it/s]
Only one chain was sampled, this makes it impossible to run some convergence checks


0.9054529
[0.9458999 2.6006136]


This shows that that my code base produces the same answer as the original PyMC3 code base. 

### Running with tensorflow backend (restart kernel for a fresh import)

After restarting the kernel, I import the local PyMC3 package again, but now I set the backend to be theano.

In [1]:
import sys
import os
import numpy as np
sys.path.append('./pymc3-master/')
os.environ["PYMC_SYMB_BACKEND"] = "tensorflow" 
import pymc3 as pm

Using TensorFlow backend.
/Users/chris/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Initialize random number generator
np.random.seed(123)

# True parameter values
alpha, sigma = 1, 1; beta = [1, 2.5]

# Predictor variable
X1 = np.random.randn(100)
X2 = np.random.randn(100) * 0.2

# Simulate outcome variable
Y = alpha + beta[0]*X1 + beta[1]*X2 + np.random.randn(100)*sigma
Y=Y.astype('float32')

In [3]:
basic_model2 = pm.Model()
with basic_model2:
    
    # requires initial value now. 
    # also has to be float at the moment. 
    alpha = pm.Normal(name='alpha',initial_value=0.0, mu=0.0, sd=10.0) 
    beta = pm.Normal(name='beta',initial_value=np.array([1.0,1.0],dtype='float32'), mu=0.0, sd=10.0, shape=2)

    mu = alpha + beta[0]*X1 + beta[1]*X2

    Y_obs = pm.Normal(name='Y_obs',initial_value=np.ones_like(Y), mu=mu, sd=1.0, observed=Y)

    step = pm.Metropolis()
    trace = pm.sample(5000,step=step,cores=1,chains=1)

print(np.mean(trace['alpha:0']))
print(np.mean(trace['beta:0'],axis=0))

Sequential sampling (1 chains in 1 job)
CompoundStep
>Metropolis: [<tf.Variable 'alpha:0' shape=() dtype=float32_ref>]
>Metropolis: [<tf.Variable 'beta:0' shape=(2,) dtype=float32_ref>]
100%|██████████| 5500/5500 [00:12<00:00, 455.22it/s]
Only one chain was sampled, this makes it impossible to run some convergence checks


0.905485
[0.947812 2.62094 ]


It's a bit slower, but it still works! Notice the variable names are `tf.Variables` now. The answer is virtually the same. One reason for the slight differences may be that I set the TensorFlow variables to be float32's vs float64's for Theano. 
